This notebook is a proof of concept for an algorithim that takes as  input the symbol of a US exchange-listed company and retrieves the content of the first section:  "Item 1. - Business", from all of the company's available 10-K files in and returns a JSON file containing the elements in the following format: {"year": "<ITEM-1-HTML-ELEMENT>"}
    
This proof of creates that desired output for Tesla, using a hand created local file that contains the links to recent 10-K fillings for Tesla. To complete this script as desired would require a method that given a stock symbol can create a file of the form:

File is named "company_symbol".txt, file contains lines of:
"year SEC_url_for_10k_for_year.htm"

Because of the limitations of the sec-api, the sec url's are the only imput that can be used to parse for item 1 (not local files downloaded with sec_edgar_downloader). This is why I have created a proof of concept not the full script. In my "Scrapbook" notebook, there is the begging of a method to query the htm links (using Lucene query syntax) which could potentially be expanded upon to validate the proof of concept.
    
This script utilizes sec-api.io to do the parsing the 10-k for the HTML item 1. I have purchased a 1 - month startup license for the purpose of this project. 

In [81]:
# import json library
import json

#import API
from sec_api import ExtractorApi

# authenticate API key
extractorApi = ExtractorApi("b33c5215f9268b5f8356340e5452cff0e52d23d9a52d01e5a956409948f60bb6")

In [88]:
# call to desired json file

runScript("TSLA")

In [86]:
# will write JSON file of {year1: HTML object, year2: HTML object}
# file will be named in format of company_symbol.json
# will utilize several helper funcitons

def runScript(ticker):
    fileName = ticker + ".txt"
    dictionary = createDictionary(fileName)
    writeJSON(dictionary, ticker)
    

In [84]:
# given a local file of the format line1 = year sec_filling_for_year.htm
# this function will return a python dictionary format {"year":"HTML-item"}

def createDictionary(fileName):
    dictionary = {}
    with open(fileName, "r") as file:
        for line in file:
            temp = line.split(" ")
            year = str(temp[0])
            url = str(temp[1])
            item1HTML = getHTML(url)
            dictionary.update({year:item1HTML})
    return dictionary

In [83]:
# re-import API
from sec_api import ExtractorApi

# re-authenticate API key
extractorApi = ExtractorApi("b33c5215f9268b5f8356340e5452cff0e52d23d9a52d01e5a956409948f60bb6")

# given a url to the .htm website for a 10-k, returns html object
# this html object represents item 1 in the given 10-k

def getHTML(url):
    item1HTML = extractorApi.get_section(url, "1", "html")
    return(item1HTML)

In [85]:
# re-import json
import json

# this function takes parameter dictionary {"year":"HTML-object"}, company symbol
# it creates a json object file titled "symbol.json", representing the dictionary

def writeJSON(dictionary, ticker):
    newFile = ticker + ".json"
    with open(newFile,"w") as file:
        json.dump(dictionary, file, indent = 4)
